In [1]:
import tensorflow as tf
import numpy as np

In [5]:
class CNNModel(object):
    def __init__(self, batch_size = 50, learning_rate = 1e-4, dropout_rate = 0.5):
        # 因为最后的节点数受stride的影响，具体结构牵一发而懂全身，所以层数和节点数不作为输入设置
        # 在这个例子中，图片初始为28*28，cnn的stride为[1, 1, 1, 1]图片大小维持不变，pool的stride为[1, 2, 2, 1]，导致缩水一半，
        # 全部经历2个cnn和2个pool，所以最终得到的图片为7*7.
        self.batch_size = batch_size
        self.learning_rate = learning_rate # 如果需要调整的话，应该使用tf.Variable
        self.dropout_rate = dropout_rate 
        self.global_step = tf.Variable(0, trainable = False)
        
        def _w_variable(shape):
            tmp = tf.truncated_normal(shape, stddev = 0.1)
            return tf.Variable(tmp)
        def _b_variable(shape):
            tmp = tf.constant(0.1, shape = shape)
            return tf.Variable(tmp)
        def _conv2d(x, w):
            return tf.nn.conv2d(x, w, strides = [1, 1, 1, 1], padding = "SAME")
        def _max_pool_2X2(x):
            return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")
        def _cnn_model(input_images, dropout_rate = 0.5):
            # 一个in_channel到一个out_channel只能由一种特征提取，
            # filter的4D分别表示[filter_height, filter_width, in_channel, out_channel]
            # 注意和input的区别：[batch, in_height, in_width, in_channels]这里是[1, 28, 28, 1]
            w_conv1 = _w_variable([5, 5, 1, 32])
            w_conv2 = _w_variable([5, 5, 32, 64])
            w_fc1 = _w_variable([7*7*64, 1024])
            w_fc2 = _w_variable([1024, 10])
            b_conv1 = _b_variable([32])
            b_conv2 = _b_variable([64])
            b_fc1 = _b_variable([1024])
            b_fc2 = _b_variable([10])
            # 让我感到奇怪的是，tf.nn.conv2d的输出怎么会是[32]类型的，不是same type as input吗，
            # 所以他应该是[1, 28, 28, 32]吧。那么他和b_conv1是怎么相加的。回去看一下tensor相加，也许能够找到答案
            h_conv1 = tf.nn.relu(_conv2d(input_images, w_conv1) + b_conv1)
            h_pool1 = _max_pool_2X2(h_conv1)
            h_conv2 = tf.nn.relu(_conv2d(h_pool1, w_conv2) + b_conv2)
            h_pool2 = _max_pool_2X2(h_conv2)
            # reshape然后接两个全连接层，然后输出，中间加一个dropout以避免Overfitting, 为什么不softmax呢？？
            # relu的输出应该不保证小于1的说。 可结果表明效果不错，为什么？？？
            # 是下面算loss的算法里面，调用了吗？？
            h_pool2_flat = tf.reshape([-1, 7 * 7 * 64])
            h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)
            h_fc1_drop = tf.nn.dropout(h_fc1, dropout_rate)
            y_conv = tf.nn.relu(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)
            return y_conv
        # 核心模型已经有了，接下来是设置饲料，包括image和label
        # 为什么要用tf.float类型的呢数据吗
        input_data = tf.placeholder(tf.float32, [None, 28 * 28])
        input_images = tf.reshape(input_data, [-1, 28, 28, 1])
        input_labels = tf.placeholder(tf.float32, [None, 10])
        # 然后用cross_entropy计算loss, 用adamOptimizer进行优化
        y_conv = _cnn_model(input_images)
        self.cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(label = input_labels, logits = y_conv))
        self.train_step = tf.train.AdamOptimizer(self.learning_rate).minimize(cross_entropy)
        correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(input_labels, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) # 一次就一个结果，怎么mean？？？
        # set global_step
        tmp_step = tf.add(self.global_step, tf.constant(1))
        update = tf.assign(self.global_step, tmp_step)
        # 获取saver
        self.saver = tf.train.Saver(tf.global_variables())
    
    def step(self, session, input_data, input_labels):
        # 用给定的输入，训练模型
        count = len(input_data)
        for _ in range(count - 1):
            session.run(self.train_step, feed_dict = {input_data : input_data, input_labels : input_labels})
            session.run(update)
        session.run(update)
        return session.run(self.cross_entropy, feed_dict = {input_data: input_data, input_labels: input_labels})
        
    def evaluate(self, session, input_data, input_labels):
        return self.accuracy.eval(feed_dict = {input_data : input_data, input_labels : input_labels}, session = session)
    

In [ ]:
# 还差了一个getbatch() 写到runner里面了